# 16장. 코루틴

이 장에서는 다음 내용들을 설명한다.

* 제너레이터를 코루틴으로 만드는 방법
* 코루틴으로 작동하는 제너레이터의 동작과 상태
* 제너레이터 객체의 `close()`와 `throw()` 메서드를 통해 호출자가 코루틴을 제어하는 방법
* 종료할 때 코루틴이 값을 반환하는 방법
* 새로운 `yield from` 구문의 사용법과 의미
* 사용 예: 시뮬레이션의 동시 활동을 관리하기 위한 코루틴

## 16.1 코루틴은 제너레이터에서 어떻게 진화했는가?

제너레이터가 호출자에 데이터를 생성해주고 호출자로부터 데이터를 받으면서 호출자와 협업하는 프로시저를 코루틴이라고 한다.

## 16.2 코루틴으로 사용되는 제너레이터의 기본 동작

다음 예제는 코루틴의 동작을 보여준다.

In [1]:
def simple_coroutine(): # 코루틴은 본체 안에 yield문을 가진 일종의 제너레이터 함수로 정의된다
    print('-> coroutine started')
    x = yield # yield문을 표현식에 사용한다.
    print('-> coroutine received:', x)

In [2]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x7f230af98b10>

In [3]:
next(my_coro) # 제너레이터를 yield문까지 실행함으로써 데이터를 전송할 수 있는 상태를 만든다.

-> coroutine started


In [4]:
my_coro.send(42) # send() 메서드를 호출해서 본체 안의 yield문의 값을 42로 만든다.
# 이제 코루틴이 실행되어서 다음 yield문이 나오거나 종료될 때까지 실행된다.

-> coroutine received: 42


StopIteration: 

제어 흐름이 코루틴 본체의 끝에 도달하므로 일반적인 제너레이터와 마찬가지로 `StopIteration` 예외를 발생시킨다.

코루틴은 다음 네 가지 상태를 가지며, `inspect.getgeneratorstate()` 함수를 이용해 현재 상태를 알 수 있다.

* `GEN_CREATED` : 실행을 위해 대기하고 있는 상태
* `GEN_RUNNING` : 현재 인터프리터가 실행하고 있는 상태
* `GEN_SUSPENDED` : 현재 yield 문에서 대기하고 있는 상태
* `GEN_CLOSED` : 실행이 완료된 상태

`send()` 메서드에 전달한 인수가 대기하고 있는 yield 표현식의 값이 되므로, 코루틴이 대기 상태에 있을 때는 `my_coro.send(42)`와 같은 형태로만 호출할 수 있다. 그러나 코루틴이 아직 기동되지 않은 `GEN_CREATED` 상태인 경우에는 `send()` 메서드를 호출할 수 없다. 그래서 코루틴을 처음 활성화하기 위해 `next(my_coro)`를 호출한다.

코루틴 객체를 생성한 직후 `None`이 아닌 값을 전달하려고 하면 다음과 같이 오류가 발생한다.

In [11]:
my_coro = simple_coroutine()
my_coro.send(1729)

TypeError: can't send non-None value to a just-started generator

처음 `next(my_coro)`를 호출할 때 코루틴을 기동한다. 즉 처음 yield문까지 실행을 진행한다. 

yield문이 두 번 이상 나오는 예제를 보자.

In [12]:
def simple_coro2(a):
    print('-> Started: a=', a)
    b = yield a
    print('-> Received: b=', b)
    c = yield a + b
    print('-> Received: c=', c)

In [21]:
my_coro2 = simple_coro2(14)

In [22]:
from inspect import getgeneratorstate
getgeneratorstate(my_coro2) # 아직 실행안됨

'GEN_CREATED'

In [23]:
next(my_coro2) # 첫번째 yield문까지 진행하면서 메세지를 출력하고 a의 값을 생성한 후 b에 값이 할당될 때까지 기다린다.

-> Started: a= 14


14

In [24]:
getgeneratorstate(my_coro2) # 코루틴이 yield문에서 대기하고 있는 상태

'GEN_SUSPENDED'

In [25]:
my_coro2.send(28) # 중단된 코루틴에 28을 보내면 yield문의 값은 28로 평가되고 이 값이 b에 할당된다. 그후 a+b의 값(42)가 생성된다.

-> Received: b= 28


42

In [26]:
my_coro2.send(99) # yield는 99로 평가되고 코루틴은 종료된다.

-> Received: c= 99


StopIteration: 

In [27]:
getgeneratorstate(my_coro2) # 종료된 상태

'GEN_CLOSED'

코루틴의 할당문에서는 실제 값을 할당하기 전에 = 오른쪽 코드를 실행한다. 즉 `b = yield a`에서는 나중에 호출자가 값을 보낸 후에야 변수 b가 설정된다. 

## 16.3 예제: 이동 평균을 계산하는 코루틴

약간 더 복잡한 코루틴 예제를 살펴보자.

In [31]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True: 
        # 이 코루틴은 호출자가 close() 메서드를 호출하거나 
        # 객체에 대한 참조가 모두 사려저서 가비지 컬렉트되어야 종료됨
        term = yield average
        total += term
        count += 1
        average = total/count

코루틴을 사용하면 total과 count를 지역 변수로 사용할 수 있다.

In [36]:
coro_avg = averager()
next(coro_avg) # average의 초깃값 None 반환

In [33]:
coro_avg.send(10)

10.0

In [34]:
coro_avg.send(30)

20.0

In [35]:
coro_avg.send(5)

15.0

## 16.4 코루틴을 기동하기 위한 데커레이터

코루틴은 반드시 `next(my_coro)`를 호출해야하는데, 이를 편리하게 사용할 수 있도록 기동하는 데커레이터가 종종 사용된다. 대표적으로 `@coroutine`이 사용된다.

* 일반적인 데커레이터는 키워드 인수를 지원하지 않으며, 데커레이트된 함수의 `__name__`과 `__doc__` 속성을 가린다. 이를 functools.wraps() 데커레이터를 이용해서 해결할 수 있다.

In [37]:
# coroutil.py

from functools import wraps

def coroutine(func):
    """데커레이터: 'func'를 기동해서 첫번째 yield까지 진행한다."""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen) # 제너레이터를 기동한다.
        return gen # 제너레이터를 반환한다.
    return primer

In [38]:
from coroutil import coroutine

@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

이제 돌려보자

In [40]:
coro_avg = averager()
from inspect import getgeneratorstate
getgeneratorstate(coro_avg) # 코루틴이 값을 받을 준비가 되어 있다.

'GEN_SUSPENDED'

In [41]:
coro_avg.send(10) # coro_avg 객체에 바로 값을 전송할 수 있다.

10.0

In [42]:
coro_avg.send(30)

20.0

In [43]:
coro_avg.send(5)

15.0

`yield from` 구문은 자동으로 자신을 실행한 코루틴을 기동시키므로 `@coroutine` 데커레이터와 함께 사용할 수 없다.

## 16.5 코루틴 종료와 예외 처리

코루틴 안에서 발생한 예외를 처리하지 않으면, `next()`나 `send()`로 코루틴을 호출한 호출자에 예외가 전파된다.

In [44]:
from coroaverager1 import averager
coro_avg = averager()
coro_avg.send(40)

40.0

In [46]:
coro_avg.send(50)

45.0

In [47]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [49]:
coro_avg.send(60) # 코루틴 안에서 예외를 처리하지 않으므로 코루틴이 종료됨

StopIteration: 

위 예제처럼 코루틴에 구분 표시를 전송해서 코루틴을 종료할 수 있다. 

제너레이터 객체는 호출자가 코루틴에 명시적으로 예외를 전달할 수 있게 해주는 `throw()`와 `close()` 메서드를 제공한다.

* generator.throw(exc_type[, exc_value[, traceback]])
  : 제너레이터가 중단한 곳의 yield 표현식에 예외를 전달한다. 제너레이터가 예외를 처리하면 제어 흐름이 다음 yield문까지 진행하고, 생성된 값은 `generator.throw()` 호출 값이 된다. 제너레이터가 예외를 처리하지 않으면 호출자까지 예외가 전파된다.

* generator.close()
  : 제너레이터가 실행을 중단한 yield 표현식이 GeneratorExit 예외를 발생시키게 만든다. 제너레이터가 예외를 처리하지 않거나 StopIteration 예외를 발생시키면 아무러 ㄴ에러도 호출자에 전달되지 않는다.

In [5]:
# coro_exc_demo.py

class DemoException(Exception):
    """설명에 사용할 예외 유형"""

def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else: # 예외가 발생하지 않으면 받은 값을 출력한다.
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.') # 이 코드는 실행되지 않음
    # 무한루프는 처리되지 않은 예외에서만 중지될 수 있으며
    # 예외가 처리되지 않으면 코루틴의 실행이 바로 중지되기 때문

이제 예외가 처리되므로 `demo_exc_handling()` 코루틴은 계속 실행된다.

In [51]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [52]:
exc_coro.send(11)

-> coroutine received: 11


In [53]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [54]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

처리되지 않는 예외를 더니면 코루틴이 중단된다.

In [55]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [56]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

코루틴이 어떻게 종료되든 어떤 정리 코드를 실행해야 하는 경우에는 `try/finally` 블록 안에 해당 코드를 넣어야 한다.

In [13]:
# coro_finally_demo.py

class DemoException(Exception):
    """설명에 사용할 예외 유형"""

def demo_finally():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else: # 예외가 발생하지 않으면 받은 값을 출력한다.
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

In [14]:
exc_coro = demo_finally()
next(exc_coro)

-> coroutine started


In [15]:
exc_coro.send(11)

-> coroutine received: 11


In [16]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [17]:
exc_coro.throw(ZeroDivisionError) # 에러가 발생했지만 출력은 됨

-> coroutine ending


ZeroDivisionError: 

## 16.6 코루틴에서 값 반환하기

코루틴은 값을 반환할 수도 있다.

In [58]:
# coroaverager2.py

from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break # 값을 반환하려면 코루틴이 정상적으로 종료되어야 한다.
            # 따라서 이 averager 버전에서는 루프를 빠져나오는 조건을 검사한다.
        total += term
        count += 1
        average = total/count
    return Result(count, average) # nametuple 반환

In [67]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10) # 값을 생성하지 않음

In [68]:
coro_avg.send(30)

In [69]:
coro_avg.send(6.5)

In [70]:
coro_avg.send(None) # None을 보내면 루프를 빠져나오고 코루틴이 결과를 반환하면서 종료
# 예외 객체의 value 속성에는 반환된 값이 들어있다.

StopIteration: Result(count=3, average=15.5)

코루틴이 반환한 값을 가져오려면 다음과 같이 하면 된다.

In [71]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10) # 값을 생성하지 않음

In [72]:
coro_avg.send(30)

In [73]:
coro_avg.send(6.5)

In [74]:
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value

In [75]:
result

Result(count=3, average=15.5)

## 16.7 yield from 사용하기

`yield from`은 완전히 새로운 언어 구성체이다. 제너레이터 `gen()`이 `yield from subgen()`을 호출하고, `subgen()`이 이어받아 값을 생성하고 `gen()`의 호출자에 반환한다.

14장에서 `yield from`을 for 루프 안의 yield에 대한 단축문으로 사용할 수 있다고 설명했다.

In [77]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i

In [78]:
list(gen())

['A', 'B', 1, 2]

위 코드는 다음과 같이 바꿀 수 있다.

In [79]:
def gen():
    yield from 'AB'
    yield from range(1,3)
    
list(gen())

['A', 'B', 1, 2]

앞에서 나왔던 예제는 다음과 같았다.

In [80]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [81]:
s = 'ABC'
t = tuple(range(3))
list(chain(s,t))

['A', 'B', 'C', 0, 1, 2]

`yield from x` 표현식이 x 객체에 대해 첫 번째로 하는 일은 `iter(x)`를 호출해서 x의 반복자를 가져오는 것이다. 그러나 `yield from`의 주요한 특징은 가장 바깥쪽 호출자와 가장 안쪽에 있는 하위 제너레이터 사이에 양방향 채널을 열어준다는 것이다.

`yield from`을 사용하기 위한 주요 용어는 다음과 같다.

* 대표 제너레이터
  : `yield from <반복형>` 표현식을 담고 있는 제너레이터 함수
* 하위 제너레이터
  : `yield from` 표현식 중 <반복형>에서 가져오는 제너레이터
* 호출자
  : 대표 제너레이터를 호출하는 코드

다음 스크립트는 가상의 중1 남학생과 여학생의 몸무게와 키를 담은 딕셔너리를 읽는다. 예를 들어 `boys;m` 키는 남학생 9명의 키에 매핑되고, `girls;kg` 키는 여학생 10명의 몸무게에 매핑된다.

In [91]:
# coroaverager3.py

from collections import namedtuple

Result = namedtuple('Result', 'count average')

# 하위 제너레이터
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield # main() 안의 클라이언트가 전송하는 값이 여기에 바인딩
        if term is None: # 종료 조건
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average) #반환값은 groper()의 yield from의 값이 된다

# 대표 제너레이터
def grouper(results, key):
    while True: # 반복때마다 하나의 averager() 객체 생성
        # grouper()가 값을 받을 때마다 이 값은 yield from에 의해 averager() 객체로 전달된다
        # grouper()는 클라이언트가 전송한 값들을 averager()가 소진할 때까지 여기에서 중단
        # averager() 객체가 실행을 완료하고 반환한 값은 results[key]에 바인딩
        results[key] = yield from averager()
        
# 호출자
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key) # 코루틴 생성
        next(group) # 코루틴 기동
        for value in values:
            group.send(value) # 값을 하나씩 grouper()에 전달
            # 이 값은 averager()의 term = yield 값이 된다
        group.send(None) # 이 부분이 중요!
        # None을 전달하면 현재 avergaer() 객체가 종료되고 grouper()가 실행을 재개
        # 여기가 없으면 스크립트는 아무것도 출력하지 않는다
    
    print(results)
    report(results)

# 실행 결과 보고서
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
            result.count, group, result.average, unit))
        
data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
}
        
if __name__ == '__main__':
    main(data)

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


In [90]:
!python3 coroaverager3.py

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


작동과정은 다음과 같다.

* 바깥쪽 for 루프를 반복할 때마다 group이라는 이름의 `grouper()` 객체를 새로 생성한다.
* `next(group)`을 호출해서 `grouper()` 대표 제너레이터를 기동시킨다. 그러면 하위 제너레이터 `averager()`를 호출한 후 `yield from`에서 대기한다.
* 내부 for 루프에서 `group.send(value)`를 호출해서 `averager()`에 데이터 전달
* 내부 for 루프가 끝났을 때 `grouper()` 객체는 여전히 `yield from`에 멈춰있으므로 `grouper()` 본체 안의 `results[key]`에 대한 할당은 아직 실행되지 않는다.
* 바깥쪽 for 루프에서 마지막으로 `group.send(None)`을 호출하지 않으면, 하위 제너레이터인 `averager()`의 실행이 종료되지 않으므로, 대표 제너레이턴 group이 다시 활성화되지 않고 결국 `results[key]`에 아무런 값도 할당되지 않는다.
* 바깥쪽 for 루프가 다시 반복되면 새로운 grouper() 객체가 생성되며 기존 객체는 가비지 컬렉트 된다.

모든 `yield from` 체인은 가작 바깥쪽 대표 제너레이터에 `next()`와 `send()`를 호출하는 클라이언트에 의해 주도된다. 

## 16.8 yield from의 의미

`yield from`의 동작은 다음과 같다.

* 하위 제너레이터가 생성하는 값은 모두 대표 제너레이터의 호출자에 바로 전달
* `send()`를 통해 대표 제너레이터에 전달한 값은 모두 하위 제너레이터에 직접 전달된다. 값이 `None`이면 하위 제너레이터의 `__next__()` 메서드가 호출된다. `None`이 아니면 하위 제너레이터의 `send()` 메서드가 호출된다. 호출된 메서드에서 `StopIteration` 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터 전달된다.
* 제너레이터나 하위 제너레이터에서 `return expr` 문을 실행하면 제너레이터를 빠져나온 후 `StopIteration(expr)` 예외가 발생한다.
* 하위 제너레이터가 실행을 마친 후 발생한 `StopIteration` 예외의 첫 번째 인수가 `yield from` 표현식의 값이 된다.
* 대표 제너레이터에 던져진 `GeneratorExit` 이외의 예외는 하위 제너레이터의 `throw()` 메서드에 전달된다. `throw()` 메서드를 호출해서 `StopIteration` 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터에 전달된다.
* `GeneratorExit` 예외가 대표 제너레이터에 던져지거나 대표 제너레이터의 `close()` 메서드가 호출되면 하위 제너레이터의 `close()` 메서드가 호출된다. 그 결과 예외가 발생하면 발생한 예외가 대표 제너레이터에 전파된다. 그렇지 않으면 대표 제너레이터에서 `GeneratorExit` 예외가 발생한다.

```python
RESULT = yield from EXPR
```
의 실행 결과는 다음과 동일하다.

```python
_i = iter(EXPR) # 반복형을 EXPR로 사용 가능
try:
    _y = next(_i) # 하위 제너레이터 가동
except StopIteration as _e:
    _r = _e.value # StopIteration이 발생하면 value가 RESULT의 값이 된다.
else:
    while 1: # 이 루프 동안은 대표 제너레이터의 실행이 중단된다.
        _s = yield _y # 하위 제너레이터에서 생성한 값을 그대로 생성하고, 호출자가 보낼 _s를 기다린다.
        try:
            _y = _i.send(_s) # 호출자가 보낸 _s를 하위 제너레이터에 전달
        except StopIteration as _e: # StopIteration 발생하면 루프 빠져나온 후 대표 제너레이터 실행 재개
            _r = _e.value
            break
RESULT = _r
```

예외 처리는 투머치...

## 16.9 사용 사례: 이산 이벤트 시뮬레이션을 위한 코루틴

이 절에서는 코루틴과 표준 라이브러리 객체만 이용해서 시뮬레이션을 구현한다.

### 16.9.1 이산 이벤트 시뮬레이션에 대해

이산 이벤트 시뮬레이션(DES)은 시스템을 일련의 이벤트로 모델링한다. 턴제 게임을 생삭하면 된다.

### 16.9.2 택시 집단 시뮬레이션

구현할 시뮬레이션 프로그램에서는 먼저 아주 많은 택시를 생성한다. 각 택시는 일정 횟수의 운행을 마친 후 집으로 돌아간다. 택시는 차고를 나와 승객을 찾으면서 배회한다. 이 상태는 승객을 태울 때까지 게속되며, 그러고 나서 운행이 시작된다. 승객이 내리고 나면 택시는 다시 배회 상태로 돌아간다. 택시가 배회하고 운행하는 시간은 지수 분포를 이용해서 생성한다. 각 상태의 변화는 이벤트로 나타낸다.

코드의 핵심은 코루틴인 `taxi_process()`와 핵심 루프를 실행하는 `Simulator.run()`이다. 이벤트는 `nametuple`을 이용하여 다음과 같이 정의된다.

In [18]:
import collections
Event = collections.namedtuple('Event', 'time proc action')

In [19]:
def taxi_process(ident, trips, start_time=0):
    """각 단계 변화마다 이벤트를 생성하며 시뮬레이터에 제어권을 넘긴다."""
    
    # 'leave garbage' 이벤트 생성 후 코루틴이 중단 
    # 이후 재개될 때 핵심 루프는 time에 할당된 현재 시간을 보내준다.
    time = yield Event(start_time, ident, 'leave garbage')
    
    for i in range(trips): # 각 운행마다 이 블록이 한번씩 반복 실행
        time = yield Event(time, ident, 'pick up passenger') # 승객 태우는 이벤트
        time = yield Event(time, ident, 'drop off passenger') # 승객 내리는 이벤트
        
    yield Event(time, ident, 'going home') # 주어진 횟수만큼 운행한 후에 집 가는 이벤트 생성
    # 여기서는 시간을 사용하지 않을 것이므로 시간을 저장하지 않음
    
    #택시 프로세스의 끝 > StopIteration 예외 발생

택시 운행해보자.

In [20]:
taxi = taxi_process(ident=13, trips=2, start_time=0)
next(taxi) # 코루틴 기동

Event(time=0, proc=13, action='leave garbage')

In [21]:
taxi.send(_.time + 7) # _ 변수는 마지막 결과값에 바인딩

Event(time=7, proc=13, action='pick up passenger')

In [22]:
taxi.send(_.time + 23) # 운행하는데 23분 걸림

Event(time=30, proc=13, action='drop off passenger')

In [23]:
taxi.send(_.time + 5) # 택시가 5분 배회한 후 승객 태운다

Event(time=35, proc=13, action='pick up passenger')

In [24]:
taxi.send(_.time + 48) # 마지막 운행은 48분 걸린다.

Event(time=83, proc=13, action='drop off passenger')

In [25]:
taxi.send(_.time + 1) # 운행 마친 후 집가는 이벤트 발생

Event(time=84, proc=13, action='going home')

In [26]:
taxi.send(_.time + 18) # 코루틴의 실행 완료

StopIteration: 

`Simlator` 클래스의 객체를 생성하기 위해 `main()` 함수는 다음과 같이 `taxis` 딕셔너리를 만든다.

```python
taxis = {i: taxi_process(i, (i+1)*2, i * DEPARTURE_INTERVAL)
        for i in range(num_taxis)}
sim = Simulator(taxis)
```

`Simulator` 클래스의 `__init__()` 메서드는 다음 두 구조체를 이용하여 정의된다.

* self.events : Event 객체를 담고 있는 `PriorityQueue` 객체. 항목을 넣고 나서 정렬된 순서대로 꺼내온다.
* self.procs : 각 프로세스(택시를 나타내는 제너레이터 객체) 번호를 시뮬레이션의 활성화된 프로세스로 매핑한다.

```python
class Simulator:
    def __init__(self, procs_map):
        self.events = queue.PriorityQueue() # 예정된 이벤트를 시간순으로 정렬해서 보관
        self.procs = dict(procs_map) # 클라이언트가 전달한 객체를 변경하지 않기 위해 사본 보관
```

우선순위 큐는 이산 이벤트 시뮬레이션의 핵심 기반이다. 이벤트를 무작위 순서로 만들어 큐에 넣은 후, 예정된 시각순으로 이벤트를 꺼내야 하기 때문이다. 

시뮬레이션의 핵심 알고리즘인 `Simulator.run()` 메서드를 살펴보자. 

1. 택시를 나타내는 프로세스를 반복

    * `next()`를 호출해서 각 택시에 대한 코루틴 기동, 첫번째 이벤트 생성    
    * `Simulator`의 `self.events` 큐에 각각의 이벤트 저장
    
1. `sim_time < end_time` 인 동안 시뮬레이션 핵심 루프를 실행한다.

    * `self.events`가 비어있으면 루프를 빠져나간다.
    * `self.event`에서 `current_event`를 가져온다. 이 객체는 `PriorityQueue`에서 가장 작은 time을 가진 Event 객체
    * Event 출력
    * current_event의 time 속성으로 시뮬레이셔 ㄴ시각 설정
    * current_event의 proc 속성으로 알아낸 코루틴에 시간 보내면 코루틴이 next_event 생성
    * next_event를 `self.event` 큐에 추가해서 스케쥴링

In [27]:
class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)

    def run(self, end_time):  # 
        """시간이 끝날 때까지 이벤트를 스케줄링하고 출력한다."""
        # 각 택시의 첫 번째 이벤트를 스케쥴링한다.
        for _, proc in sorted(self.procs.items()):  # 각 택시를 호출
            first_event = next(proc)  # 첫번째 이벤트(정거장 출발)
            self.events.put(first_event)  # 각 이벤트를 큐에 넣는다

        # 시뮬레이션 핵심 루프
        sim_time = 0  # 시뮬레이션 시계
        while sim_time < end_time:  
            if self.events.empty(): # 큐 안에 이벤트가 없으면 종료
                print('*** end of events ***')
                break

            current_event = self.events.get()  # time이 가장 작은 이벤트를 가져옴
            sim_time, proc_id, previous_action = current_event  # 현재 시각 갱신
            print('taxi:', proc_id, proc_id * '   ', current_event)  # 택시의 ID에 따라 들여씀
            active_proc = self.procs[proc_id]  # 코루틴 가져옴
            next_time = sim_time + compute_duration(previous_action)  # 다음 활성화시간 계산
            try:
                next_event = active_proc.send(next_time)  # 택시 코루틴에 시각 전송
            except StopIteration:
                del self.procs[proc_id]  # 해당 코루틴 제거
            else:
                self.events.put(next_event)  # 예외가 발생하지 않으면 next_event를 큐에 넣는다.
        else:  # 시간이 초과되어 루프 종료된 경우는 대기중인 이벤트 출력
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))
# END TAXI_SIMULATOR

In [29]:
!python example-code/16-coroutine/taxi_sim.py

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 0  Event(time=3, proc=0, action='pick up passenger')
taxi: 1     Event(time=5, proc=1, action='leave garage')
taxi: 0  Event(time=6, proc=0, action='drop off passenger')
taxi: 2        Event(time=10, proc=2, action='leave garage')
taxi: 1     Event(time=11, proc=1, action='pick up passenger')
taxi: 0  Event(time=13, proc=0, action='pick up passenger')
taxi: 1     Event(time=13, proc=1, action='drop off passenger')
taxi: 2        Event(time=15, proc=2, action='pick up passenger')
taxi: 1     Event(time=19, proc=1, action='pick up passenger')
taxi: 1     Event(time=23, proc=1, action='drop off passenger')
taxi: 0  Event(time=24, proc=0, action='drop off passenger')
taxi: 2        Event(time=24, proc=2, action='drop off passenger')
taxi: 0  Event(time=25, proc=0, action='going home')
taxi: 1     Event(time=28, proc=1, action='pick up passenger')
taxi: 2        Event(time=28, proc=2, action='pick up passenger')
taxi: 2        Even

아래는 전체 코드이다.

```python

"""
Taxi simulator
==============

Driving a taxi from the console::

    >>> from taxi_sim import taxi_process
    >>> taxi = taxi_process(ident=13, trips=2, start_time=0)
    >>> next(taxi)
    Event(time=0, proc=13, action='leave garage')
    >>> taxi.send(_.time + 7)
    Event(time=7, proc=13, action='pick up passenger')
    >>> taxi.send(_.time + 23)
    Event(time=30, proc=13, action='drop off passenger')
    >>> taxi.send(_.time + 5)
    Event(time=35, proc=13, action='pick up passenger')
    >>> taxi.send(_.time + 48)
    Event(time=83, proc=13, action='drop off passenger')
    >>> taxi.send(_.time + 1)
    Event(time=84, proc=13, action='going home')
    >>> taxi.send(_.time + 10)
    Traceback (most recent call last):
      File "<stdin>", line 1, in <module>
    StopIteration

Sample run with two cars, random seed 10. This is a valid doctest::

    >>> main(num_taxis=2, seed=10)
    taxi: 0  Event(time=0, proc=0, action='leave garage')
    taxi: 0  Event(time=5, proc=0, action='pick up passenger')
    taxi: 1     Event(time=5, proc=1, action='leave garage')
    taxi: 1     Event(time=10, proc=1, action='pick up passenger')
    taxi: 1     Event(time=15, proc=1, action='drop off passenger')
    taxi: 0  Event(time=17, proc=0, action='drop off passenger')
    taxi: 1     Event(time=24, proc=1, action='pick up passenger')
    taxi: 0  Event(time=26, proc=0, action='pick up passenger')
    taxi: 0  Event(time=30, proc=0, action='drop off passenger')
    taxi: 0  Event(time=34, proc=0, action='going home')
    taxi: 1     Event(time=46, proc=1, action='drop off passenger')
    taxi: 1     Event(time=48, proc=1, action='pick up passenger')
    taxi: 1     Event(time=110, proc=1, action='drop off passenger')
    taxi: 1     Event(time=139, proc=1, action='pick up passenger')
    taxi: 1     Event(time=140, proc=1, action='drop off passenger')
    taxi: 1     Event(time=150, proc=1, action='going home')
    *** end of events ***

See longer sample run at the end of this module.

"""

import random
import collections
import queue
import argparse

DEFAULT_NUMBER_OF_TAXIS = 3
DEFAULT_END_TIME = 180
SEARCH_DURATION = 5
TRIP_DURATION = 20
DEPARTURE_INTERVAL = 5

Event = collections.namedtuple('Event', 'time proc action')


# BEGIN TAXI_PROCESS
def taxi_process(ident, trips, start_time=0):  # <1>
    """Yield to simulator issuing event at each state change"""
    time = yield Event(start_time, ident, 'leave garage')  # <2>
    for i in range(trips):  # <3>
        time = yield Event(time, ident, 'pick up passenger')  # <4>
        time = yield Event(time, ident, 'drop off passenger')  # <5>

    yield Event(time, ident, 'going home')  # <6>
    # end of taxi process # <7>
# END TAXI_PROCESS


# BEGIN TAXI_SIMULATOR
class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)

    def run(self, end_time):  # <1>
        """Schedule and display events until time is up"""
        # schedule the first event for each cab
        for _, proc in sorted(self.procs.items()):  # <2>
            first_event = next(proc)  # <3>
            self.events.put(first_event)  # <4>

        # main loop of the simulation
        sim_time = 0  # <5>
        while sim_time < end_time:  # <6>
            if self.events.empty():  # <7>
                print('*** end of events ***')
                break

            current_event = self.events.get()  # <8>
            sim_time, proc_id, previous_action = current_event  # <9>
            print('taxi:', proc_id, proc_id * '   ', current_event)  # <10>
            active_proc = self.procs[proc_id]  # <11>
            next_time = sim_time + compute_duration(previous_action)  # <12>
            try:
                next_event = active_proc.send(next_time)  # <13>
            except StopIteration:
                del self.procs[proc_id]  # <14>
            else:
                self.events.put(next_event)  # <15>
        else:  # <16>
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))
# END TAXI_SIMULATOR


def compute_duration(previous_action):
    """Compute action duration using exponential distribution"""
    if previous_action in ['leave garage', 'drop off passenger']:
        # new state is prowling
        interval = SEARCH_DURATION
    elif previous_action == 'pick up passenger':
        # new state is trip
        interval = TRIP_DURATION
    elif previous_action == 'going home':
        interval = 1
    else:
        raise ValueError('Unknown previous_action: %s' % previous_action)
    return int(random.expovariate(1/interval)) + 1


def main(end_time=DEFAULT_END_TIME, num_taxis=DEFAULT_NUMBER_OF_TAXIS,
         seed=None):
    """Initialize random generator, build procs and run simulation"""
    if seed is not None:
        random.seed(seed)  # get reproducible results

    taxis = {i: taxi_process(i, (i+1)*2, i*DEPARTURE_INTERVAL)
             for i in range(num_taxis)}
    sim = Simulator(taxis)
    sim.run(end_time)


if __name__ == '__main__':

    parser = argparse.ArgumentParser(
                        description='Taxi fleet simulator.')
    parser.add_argument('-e', '--end-time', type=int,
                        default=DEFAULT_END_TIME,
                        help='simulation end time; default = %s'
                        % DEFAULT_END_TIME)
    parser.add_argument('-t', '--taxis', type=int,
                        default=DEFAULT_NUMBER_OF_TAXIS,
                        help='number of taxis running; default = %s'
                        % DEFAULT_NUMBER_OF_TAXIS)
    parser.add_argument('-s', '--seed', type=int, default=None,
                        help='random generator seed (for testing)')

    args = parser.parse_args()
    main(args.end_time, args.taxis, args.seed)


"""

Sample run from the command line, seed=3, maximum elapsed time=120::

# BEGIN TAXI_SAMPLE_RUN
$ python3 taxi_sim.py -s 3 -e 120
taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 0  Event(time=2, proc=0, action='pick up passenger')
taxi: 1     Event(time=5, proc=1, action='leave garage')
taxi: 1     Event(time=8, proc=1, action='pick up passenger')
taxi: 2        Event(time=10, proc=2, action='leave garage')
taxi: 2        Event(time=15, proc=2, action='pick up passenger')
taxi: 2        Event(time=17, proc=2, action='drop off passenger')
taxi: 0  Event(time=18, proc=0, action='drop off passenger')
taxi: 2        Event(time=18, proc=2, action='pick up passenger')
taxi: 2        Event(time=25, proc=2, action='drop off passenger')
taxi: 1     Event(time=27, proc=1, action='drop off passenger')
taxi: 2        Event(time=27, proc=2, action='pick up passenger')
taxi: 0  Event(time=28, proc=0, action='pick up passenger')
taxi: 2        Event(time=40, proc=2, action='drop off passenger')
taxi: 2        Event(time=44, proc=2, action='pick up passenger')
taxi: 1     Event(time=55, proc=1, action='pick up passenger')
taxi: 1     Event(time=59, proc=1, action='drop off passenger')
taxi: 0  Event(time=65, proc=0, action='drop off passenger')
taxi: 1     Event(time=65, proc=1, action='pick up passenger')
taxi: 2        Event(time=65, proc=2, action='drop off passenger')
taxi: 2        Event(time=72, proc=2, action='pick up passenger')
taxi: 0  Event(time=76, proc=0, action='going home')
taxi: 1     Event(time=80, proc=1, action='drop off passenger')
taxi: 1     Event(time=88, proc=1, action='pick up passenger')
taxi: 2        Event(time=95, proc=2, action='drop off passenger')
taxi: 2        Event(time=97, proc=2, action='pick up passenger')
taxi: 2        Event(time=98, proc=2, action='drop off passenger')
taxi: 1     Event(time=106, proc=1, action='drop off passenger')
taxi: 2        Event(time=109, proc=2, action='going home')
taxi: 1     Event(time=110, proc=1, action='going home')
*** end of events ***
# END TAXI_SAMPLE_RUN

"""

```

끗!